In [ ]:
# pip install fosforml

In [ ]:
# pip show fosforml

In [2]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()
session
# s_df = session.sql("SELECT * FROM FDC_INSIGHTS_PT.INSIGHT_SPCS_SCHEMA.HR_CHURN_LATEST1MB")
# s_df.collect()
s_df = session.sql("SELECT * FROM SPCS_AMIT.SCHEMA_SPCS_AMIT.FDC_MODEL_MONITORING_AMIT")
s_df.collect()

[Row(OBJECT_ID='ff7537e9-4654-4192-b1aa-6aeed514e504', RUN_ID='2526d5eb-cabc-47cb-be04-55b5bc8af72f', PROJECT_ID='031dba4d-7f94-40ca-a850-ea33e610521b', MODEL_NAME='MODEL_031DBA4D_7F94_40CA_A850_EA33E610521B_FDC_SKLEARNMULITCLASSMODEL_3_9_6DEC_FS', VERSION_NAME='v1', OBJECT_TYPE='model_registration', OBJECT_DATA='{\n  "build_time_metrics": [\n    {\n      "model_metric_value": [\n        {\n          "column_1": 0,\n          "column_1_counter": 0,\n          "column_2": 0,\n          "column_2_counter": 0,\n          "prediction": 14\n        },\n        {\n          "column_1": 0,\n          "column_1_counter": 0,\n          "column_2": 1,\n          "column_2_counter": 1,\n          "prediction": 0\n        },\n        {\n          "column_1": 0,\n          "column_1_counter": 0,\n          "column_2": 2,\n          "column_2_counter": 2,\n          "prediction": 0\n        },\n        {\n          "column_1": 1,\n          "column_1_counter": 1,\n          "column_2": 0,\n         

In [3]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import RandomForestRegressor
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [4]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [6]:
# import pandas as pd
# df = pd.read_csv("/notebooks/notebooks/HR_CHURN_LATEST.csv")
# snow_df = session.create_dataframe(df)

input_data_frame, test_df = s_df.randomSplit([0.75, 0.25])

In [7]:
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']
input_data_frame.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SR_NO"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECTS"  |"AVERAGE_MONTHLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT_COMPANY"  |"PROMOTION_LAST_5YEARS"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |1        |0.98                  |0.3                |7                  |220                      |3                     |1                |1               |1                        |
|0.0       |6.0           |2        |0.41                  |0.9                |6                  |284                      |10                    |1      

In [8]:
pipeline = RandomForestRegressor(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)

The version of package 'snowflake-snowpark-python' in the local environment is 1.26.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


: 

In [7]:
pred_snow_df = pipeline.predict(apply_label_encoding(test_df))

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


: 

In [ ]:
from fosforml import register_model

In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_1MB",
  description="This is a test model",
  flavour="snowflake",
    dataset_source="HR_CHURN",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="regression",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)

Calculating build time metrics

Progress: ██████████████████                                                     25.0%


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.*'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-snowpark-python' in the local environment is 1.26.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.


Calculating build time metrics

Progress: ███████████████████████████████████                                    50.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████                   75.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_9FEB4029_B86E_4724_B4E6_7016390DCFF1_FDC_SNOW_12DEC_HR_CHURN_SNOW_3_10' registered successfully."

: 

In [ ]:
# register_model(
#   model_obj=pipeline,
#   session=session,
#   snowflake_df=pred_snow_df,
#   name="HR_Churn_2_3_10",
#   description="This is a test model",
#   flavour="snowflake",
#     dataset_source="HR_CHURN",
#   dataset_name="Hr_churn",
#   datasource_name="Hr_churn",
#   model_type="regression",
#   conda_dependencies=["scikit-learn==1.3.2"],
#   source="Notebook"
# )

In [ ]:
# register_model(
#   model_obj=pipeline,
#   session=session,
#   snowflake_df=pred_snow_df,
#   name="HR_Churn_3_3_10",
#   description="This is a test model",
#   flavour="snowflake",
#     dataset_source="HR_CHURN",
#   dataset_name="Hr_churn",
#   datasource_name="Hr_churn",
#   model_type="regression",
#   conda_dependencies=["scikit-learn==1.3.2"],
#   source="Notebook"
# )

In [ ]:
# register_model(
#   model_obj=pipeline,
#   session=session,
#   snowflake_df=pred_snow_df,
#   name="HR_Churn_4_3_10",
#   description="This is a test model",
#   flavour="snowflake",
#     dataset_source="HR_CHURN",
#   dataset_name="Hr_churn",
#   datasource_name="Hr_churn",
#   model_type="regression",
#   conda_dependencies=["scikit-learn==1.3.2"],
#   source="Notebook"
# )

In [ ]:
# register_model(
#   model_obj=pipeline,
#   session=session,
#   snowflake_df=pred_snow_df,
#   name="HR_Churn_5_3_10",
#   description="This is a test model",
#   flavour="snowflake",
#     dataset_source="HR_CHURN",
#   dataset_name="Hr_churn",
#   datasource_name="Hr_churn",
#   model_type="regression",
#   conda_dependencies=["scikit-learn==1.3.2"],
#   source="Notebook"
# )